# Import

In [ ]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp61sxotst/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.3
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift


In [ ]:
import Foundation
import TensorFlow
import SwiftCoreMLTools
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [ ]:
// !wget -O "../data/housing.csv" https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
    try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
}

# Data Ingestion

In [ ]:
// TODO: Download housing.csv

let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

let trainPercentage:Float = 0.8
let upToRecord = Int(ceil(Float(numRecords) * trainPercentage))

let trainFeatures = Array(dataFeatures[0..<upToRecord])
let trainLabels = Array(dataLabels[0..<upToRecord])
let testFeatures = Array(dataFeatures[upToRecord...])
let testLabels = Array(dataLabels[upToRecord...])

print("Training shapes [\(trainFeatures.count) \(trainFeatures[0].count)] [\(trainLabels.count) \(trainLabels[0].count)]")
print("Testing shapes  [\(testFeatures.count) \(testFeatures[0].count)] [\(testLabels.count) \(testLabels[0].count)]")

let xAllTrain = Array(trainFeatures.joined())
let yAllTrain = Array(trainLabels.joined())
let xAllTest = Array(testFeatures.joined())
let yAllTest = Array(testLabels.joined())

let XTrainDeNorm = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([upToRecord, numColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([upToRecord, 1]))
let XTestDeNorm = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numRecords-upToRecord, numColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numRecords-upToRecord, 1]))

# Normalize

In [ ]:
let mean = XTrainDeNorm.mean(alongAxes: 0)
let std = XTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

In [ ]:
let XTrain = (XTrainDeNorm - mean)/std
let XTest = (XTestDeNorm - mean)/std

In [ ]:
print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

# Model

In [ ]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [ ]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [ ]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(upToRecord) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [ ]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(upToRecord, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

# Test

In [ ]:
Context.local.learningPhase = .inference

let cheat = model(XTrain)
print(cheat[0], YTrain[0])

let record: [Float] = [-0.4013274553772651,-0.48782210614046656,-1.1729760489283325,-0.2721895900613162,-0.8055945265354896,0.09156749405417394,-1.828902543802867,0.6384789935042571,-0.6351491942719604,0.1472680456555187,-0.7178137893787737,0.2073805740660824,-0.7473489168521552] 
let tfrecord = Tensor<Float>(record).reshaped(to: TensorShape([1, 13]))
let tfcheat = model(tfrecord)
print(tfcheat)

let prediction = model(XTest)
print(prediction[0], YTest[0])

let tmse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numRecords-upToRecord)
let tmae = mae(predictions: prediction, truths: YTest)/Float(numRecords-upToRecord)

print("MSE: \(tmse), MAE: \(tmae)")

# Export

In [ ]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

In [ ]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}

In [ ]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))

# Sample Record for testing Inferencing on CoreML

In [ ]:
print(XTrain[0], YTrain[0])